In [2]:
import numpy as np 

def get_Imbalance(state):
    imabalance_val =0
    state_dict = state.to_dict()

    for basis,amp in state_dict.items():
        Ne, No = 0,0
        for i in range(len(basis)):
            if i % 2 == 0 and basis[i] == '1':
                No += 1
        if Ne + No != 0:
            imabalance_val += np.abs(amp) ** 2 (Ne - No) / (Ne + No)
    return imabalance_val

<>:13: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<>:13: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
C:\Users\abira\AppData\Local\Temp\ipykernel_28172\4128953266.py:13: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  imabalance_val += np.abs(amp) ** 2 (Ne - No) / (Ne + No)


In [3]:
import matplotlib.pyplot as plt 
from qiskit import QuantumCircuit, QuantumRegister, transpile, Aer
from qiskit import ClassicalRegister
from qiskit.circuit import Parameter, Instruction
import qiskit.quantum_info as qi 
from tqdm.notebook import tqdm

In [4]:
def Trot_qc_mbl(num_qubits, t, J, deltas):
    #creates trotterized quantm circuit at given time for 1d hesenberg xxz model

    def ZZ_gate(J, t):
        ZZ_qr = QuantumRegister(2)
        ZZ_qc = QuantumCircuit(ZZ_qr, name = 'ZZ')
        ZZ_qc.cnot(0,1)
        ZZ_qc.rz(2 * J * t, 1)
        ZZ_qc.cnot(0,1)
        ZZ = ZZ_qc.to_instruction()
        return ZZ
    
    def XX_gate(t):
        XX_qr = QuantumRegister(2)
        XX_qc = QuantumCircuit(XX_qr, name = 'XX')
        XX_qc.ry(np.pi/2, [0,1])
        XX_qc.append(ZZ_gate(1,t), [0,1])
        XX_qc.ry(-np.pi/2, [0,1])
        XX = XX_qc.to_instruction()
        return XX
    def YY_gate(t):
        YY_qr = QuantumRegister(2)
        YY_qc = QuantumCircuit(YY_qr, name = 'YY')
        YY_qc.rx(-np.pi / 2, [0,1])
        YY_qc.append(ZZ_gate(1,t), [0,1])
        YY_qc.rx(np.pi/2, [0,1])
        YY = YY_qc.to_instruction()
        return YY
    
    Trot_qr = QuantumRegister(num_qubits)
    qc = QuantumCircuit(Trot_qr, name = 'Trot')

    for i in range(num_qubits -1):
        qc.append(XX_gate(t), [Trot_qr[i], Trot_qr[i+1]])
        qc.append(YY_gate(t), [Trot_qr[i], Trot_qr[i+1]])
        qc.append(ZZ_gate(J,t), [Trot_qr[i], Trot_qr[i+1]])

    for i in range(num_qubits):
        qc.rz(2 * deltas[i] *t, i)
    
    return qc

    

In [9]:
def U_trot_circuits_mbl(delta_t, trotter_steps, num_qubits, U, W, beta):
    #record list of trotterized quantumc circuits for many body
    #localization at all Trotter Steps

    t = Parameter('t')
    J = Parameter('J')
    deltas = [Parameter('delta_{:d}'.format(idx)) for idx in range(num_qubits)]

    AA_pattern = np.cos(2*np.pi*beta*np.arange(num_qubits))
    disorders = W * AA_pattern
    mbl_circuits = []

    for n_steps in trotter_steps:
        qr = QuantumRegister(num_qubits)
        cr = ClassicalRegister(num_qubits)
        qc = QuantumCircuit(qr, cr)

        qc.x([0,4,8])

        for _ in range(n_steps):
            qc.append(Trot_qc_mbl(num_qubits, t, J, deltas), qr)
        
        qc = qc.bind_parameters({t:delta_t})
        qc = qc.bind_parameters({deltas[idx]: disorders[idx] for idx in range(num_qubits)})
        qc = qc.bind_parameters({J:U})
        mbl_circuits.append(qc)

    return mbl_circuits

In [11]:
delta_t = 0.15
trotter_steps = np.arange(1,25,1)
num_qubits = 12
U =1.0
beta = (np.sqrt(5) - 1) / 2

circuits = {}
Ws = [0.2,2,4,8]

for W in Ws:
    circuits[W] = U_trot_circuits_mbl(
        delta_t= delta_t,
        trotter_steps= trotter_steps,
        num_qubits= num_qubits,
        U = U,
        W= W,
        beta = beta
    )
backend_sim = Aer.get_backend('statevector_simulator')
probability_densities = {}
state_vector_imbalances = {}
vn_entropies = {}

for W in tqdm(Ws):
    probability_densities[W] = []
    state_vector_imbalances[W] = []
    vn_entropies[W] = []

    for circ in circuits[W]:
        transpiled_circ = transpile(circ, backend_sim, optimization_level= 3)
        job_sim = backend_sim.run(transpiled_circ)
        result_sim = job_sim.result()
        outputstate = result_sim.get_statevector(transpiled_circ,decimals = 6)

        ps = []
        for idx in range(num_qubits):
            ps.append(np.abs(qi.partial_trace(outputstate, [i for i in range(num_qubits)])))

        entropy = 0
        rho_0 = qi.partial_trace(outputstate, range(1, num_qubits))
        entropy += qi.entropy(rho_0, base = np.exp(1))

        imbalance = 0
        imbalance += get_Imbalance(outputstate)

        vn_entropies[W].append(entropy)
        probability_densities[W].append(ps)
        state_vector_imbalances[W].apend(imbalance)


  0%|          | 0/4 [00:00<?, ?it/s]

C:\Users\abira\AppData\Local\Temp\ipykernel_28172\4128953266.py:13: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  imabalance_val += np.abs(amp) ** 2 (Ne - No) / (Ne + No)
C:\Users\abira\AppData\Local\Temp\ipykernel_28172\4128953266.py:13: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  imabalance_val += np.abs(amp) ** 2 (Ne - No) / (Ne + No)
C:\Users\abira\AppData\Local\Temp\ipykernel_28172\4128953266.py:13: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  imabalance_val += np.abs(amp) ** 2 (Ne - No) / (Ne + No)


TypeError: 'int' object is not callable